# Importing libraries

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn import preprocessing
from itertools import islice
import seaborn as sns

### Ignoring warnings

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
dataFrame = pd.read_csv('../data/week1_challenge.csv', na_values=['?', None])


In [5]:
#rename columns
dataFrame=dataFrame.rename(columns = {'Total DL (Bytes)' : 'totalDL','Total UL (Bytes)' : 'totalUL','Dur. (ms)' : 
                                      'dur','MSISDN/Number':'msisdn','Last Location Name':'location',
                                      'Handset Manufacturer':'manufacturer','Handset Type':'handset'
                                     })

### below I'll show the top 10 users based on session frequency

In [6]:
sessionsCountData=dataFrame['msisdn'].value_counts()
sessionsCount=sessionsCountData.values.tolist()
msisdn=sessionsCountData.index.values
sessionPerUserDictionary = dict(zip(msisdn, sessionsCount))

for key,value in list(islice(sessionPerUserDictionary.items(), 10)):
    print(key,'----------',value,'Sessions')

33626320676.0 ---------- 18 Sessions
33614892860.0 ---------- 17 Sessions
33625779332.0 ---------- 17 Sessions
33659725664.0 ---------- 16 Sessions
33675877202.0 ---------- 15 Sessions
33760536639.0 ---------- 15 Sessions
33667163239.0 ---------- 13 Sessions
33786323068.0 ---------- 12 Sessions
33627080969.0 ---------- 12 Sessions
33604515716.0 ---------- 12 Sessions


### The following cell shows the top 10 users based on the total duration of all the sessions of each user

In [8]:
durationSumDF=dataFrame.groupby('msisdn')['dur'].sum()
duration = durationSumDF.values.tolist()
msisdn=durationSumDF.index.values
durationPerUser = dict(zip(msisdn, duration))

for key,value in list(islice(durationPerUser.items(), 10)):
    print(key,'----------',value,'Microseconds')

33601001722.0 ---------- 116720.0 Microseconds
33601001754.0 ---------- 181230.0 Microseconds
33601002511.0 ---------- 134969.0 Microseconds
33601007832.0 ---------- 49878.0 Microseconds
33601008617.0 ---------- 37104.0 Microseconds
33601010682.0 ---------- 253983.0 Microseconds
33601011634.0 ---------- 128360.0 Microseconds
33601011959.0 ---------- 86399.0 Microseconds
33601014694.0 ---------- 495702.0 Microseconds
33601020306.0 ---------- 124854.0 Microseconds


### The following cell shows the top 10 users based on the total duration of all the sessions of each user

In [12]:
sum_column = dataFrame["totalUL"] + dataFrame["totalDL"]
dataFrame["totalData"] = sum_column
totalDataDF=dataFrame.groupby('msisdn')['totalData'].sum()
totalData = totalDataDF.values.tolist()
msisdn=totalDataDF.index.values

totalDataUserDictionary = dict(zip(msisdn, totalData))

for key,value in list(islice(totalDataUserDictionary.items(), 10)):
    print(key,'----------',value,'Bytes')

33601001722.0 ---------- 878690574.0 Bytes
33601001754.0 ---------- 156859643.0 Bytes
33601002511.0 ---------- 595966483.0 Bytes
33601007832.0 ---------- 422320698.0 Bytes
33601008617.0 ---------- 1457410944.0 Bytes
33601010682.0 ---------- 615217221.0 Bytes
33601011634.0 ---------- 654723066.0 Bytes
33601011959.0 ---------- 332660357.0 Bytes
33601014694.0 ---------- 990132189.0 Bytes
33601020306.0 ---------- 732463761.0 Bytes


# The top 10 customers per engagement metric


In [11]:
userEngagementDF=pd.DataFrame(columns=['MSISDN' ,'Session Frequency','Session Duration','Total Data'])
userEngagementDF['MSISDN']=durationPerUser.keys()

userEngagementDF['Session Frequency']=userEngagementDF['MSISDN'].map(sessionPerUserDictionary)
userEngagementDF['Session Duration']=userEngagementDF['MSISDN'].map(durationPerUser)
userEngagementDF['Total Data']= userEngagementDF['MSISDN'].map(totalDataUserDictionary)

userEngagementDF.sample(10)

,MSISDN,Session Frequency,Session Duration,Total Data
85539,3.369851e+10,2,103128.0,7.277828e+08
58556,3.366506e+10,1,97140.0,4.813763e+08
90561,3.375090e+10,2,26310.0,4.293895e+08
37313,3.365978e+10,3,259197.0,1.141202e+09
70361,3.366888e+10,1,86399.0,2.484075e+08
60657,3.366562e+10,1,94864.0,8.932899e+08
37107,3.365974e+10,4,281926.0,1.631017e+09
11102,3.362241e+10,1,31645.0,8.314510e+08
50222,3.366283e+10,1,33919.0,7.456491e+08
12112,3.362406e+10,1,86395.0,5.188775e+08


### a k-means (k=3) to classify customers in three groups of engagement.
